In [0]:
# -*- coding: utf-8 -*-
"""EsquemaParte3.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Ge_Pw-txgQ1PRJTdJPioBOl02yILeVuf
"""

#########################################################################
################### OBTENER LA BASE DE DATOS ############################
#########################################################################

# Descargar las imágenes de http://www.vision.caltech.edu/visipedia/CUB-200.html
# Descomprimir el fichero.
# Descargar también el fichero list.tar.gz, descomprimirlo y guardar los ficheros
# test.txt y train.txt dentro de la carpeta de imágenes anterior. Estos 
# dos ficheros contienen la partición en train y test del conjunto de datos.

##### EN CASO DE USAR COLABORATORY
# Sube tanto las imágenes como los ficheros text.txt y train.txt a tu drive.
# Después, ejecuta esta celda y sigue las instrucciones para montar 
# tu drive en colaboratory.

from google.colab import drive
drive.mount('/content/drive')
!unzip /content/drive/My\ Drive/VC/P2/images.zip -d /content/

#########################################################################
################ CARGAR LAS LIBRERÍAS NECESARIAS ########################
#########################################################################

# Terminar de rellenar este bloque con lo que vaya haciendo falta

# Importar librerías necesarias
import numpy as np
import matplotlib.pyplot as plt 
import keras
import keras.utils as np_utils

from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Activation, Input
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import load_img, img_to_array 


# Importar el optimizador a usar
from keras.optimizers import SGD

# Importar modelos y capas específicas que se van a usar

# Preprocesamiento de datos
from keras.preprocessing.image import ImageDataGenerator

# Importar el modelo ResNet50 y su respectiva función de preprocesamiento,
# que es necesario pasarle a las imágenes para usar este modelo
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input


#########################################################################
################## FUNCIÓN PARA LEER LAS IMÁGENES #######################
#########################################################################

# Dado un fichero train.txt o test.txt y el path donde se encuentran los
# ficheros y las imágenes, esta función lee las imágenes
# especificadas en ese fichero y devuelve las imágenes en un vector y 
# sus clases en otro.

def leerImagenes(vec_imagenes, path):
  clases = np.array([img.split('/')[0] for img in vec_imagenes])
  imagenes = np.array([img_to_array(load_img(path + "/" + img, 
                                             target_size = (224, 224))) 
                       for img in vec_imagenes])
  return imagenes, clases

#########################################################################
############# FUNCIÓN PARA CARGAR EL CONJUNTO DE DATOS ##################
#########################################################################

# Usando la función anterior, y dado el path donde se encuentran las
# imágenes y los archivos "train.txt" y "test.txt", devuelve las 
# imágenes y las clases de train y test para usarlas con keras
# directamente.

def cargarDatos(path):

  # Cargamos los ficheros
  train_images = np.loadtxt(path + "/train.txt", dtype = str)
  test_images = np.loadtxt(path + "/test.txt", dtype = str)
  
  # Leemos las imágenes con la función anterior
  train, train_clases = leerImagenes(train_images, path)
  test, test_clases = leerImagenes(test_images, path)
  
  # Pasamos los vectores de las clases a matrices 
  # Para ello, primero pasamos las clases a números enteros
  clases_posibles = np.unique(np.copy(train_clases))
  for i in range(len(clases_posibles)):
    train_clases[train_clases == clases_posibles[i]] = i
    test_clases[test_clases == clases_posibles[i]] = i

  # Después, usamos la función to_categorical()
  train_clases = np_utils.to_categorical(train_clases, 200)
  test_clases = np_utils.to_categorical(test_clases, 200)
  
  # Barajar los datos
  train_perm = np.random.permutation(len(train))
  train = train[train_perm]
  train_clases = train_clases[train_perm]

  test_perm = np.random.permutation(len(test))
  test = test[test_perm]
  test_clases = test_clases[test_perm]
  
  return train, train_clases, test, test_clases

#########################################################################
######## FUNCIÓN PARA OBTENER EL ACCURACY DEL CONJUNTO DE TEST ##########
#########################################################################

# Esta función devuelve el accuracy de un modelo, definido como el 
# porcentaje de etiquetas bien predichas frente al total de etiquetas.
# Como parámetros es necesario pasarle el vector de etiquetas verdaderas
# y el vector de etiquetas predichas, en el formato de keras (matrices
# donde cada etiqueta ocupa una fila, con un 1 en la posición de la clase
# a la que pertenece y 0 en las demás).

def calcularAccuracy(labels, preds):
  labels = np.argmax(labels, axis = 1)
  preds = np.argmax(preds, axis = 1)
  
  accuracy = sum(labels == preds)/len(labels)
  
  return accuracy

#########################################################################
## FUNCIÓN PARA PINTAR LA PÉRDIDA Y EL ACCURACY EN TRAIN Y VALIDACIÓN ###
#########################################################################

# Esta función pinta dos gráficas, una con la evolución de la función
# de pérdida en el conjunto de train y en el de validación, y otra
# con la evolución del accuracy en el conjunto de train y en el de
# validación. Es necesario pasarle como parámetro el historial
# del entrenamiento del modelo (lo que devuelven las funciones
# fit() y fit_generator()).

def mostrarEvolucion(hist):

  loss = hist.history['loss']
  val_loss = hist.history['val_loss']
  plt.plot(loss)
  plt.plot(val_loss)
  plt.legend(['Training loss', 'Validation loss'])
  plt.show()

  acc = hist.history['acc']
  val_acc = hist.history['val_acc']
  plt.plot(acc)
  plt.plot(val_acc)
  plt.legend(['Training accuracy', 'Validation accuracy'])
  plt.show()

################################################################################
 ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ## 
################################################################################ 

## Usar ResNet50 preentrenada en ImageNet como un extractor de características
def ExtractorCaracteristicas (x_train, y_train, x_test, y_test):

    print("Usar ResNet50 preentrenada en ImageNet como un extractor de características")

    # Preprocesamiento de los conjuntos de datos de train y test
    x_train = preprocess_input(x_train)
    x_test = preprocess_input(x_test)

    # Modelo ResNet50 (preentrenado en ImageNet y sin la última capa).
    resnet50 = ResNet50(include_top=False, weights='imagenet', pooling='avg')

    # Extraer las características las imágenes con el modelo anterior.
    train_pred = resnet50.predict(x_train, verbose=1)
    test_pred = resnet50.predict(x_test, verbose=1)

    # Las características extraídas en el paso anterior van a ser la entrada de
    # un pequeño modelo de dos capas Fully Conected, donde la última será la que 
    # nos clasifique las clases de Caltech-UCSD (200 clases). De esta forma, es 
    # como si hubiéramos fijado todos los parámetros de ResNet50 y estuviésemos
    # entrenando únicamente las capas añadidas. Definir dicho modelo.

    model = Sequential()
    model.add(Dense (2048, activation='linear', input_shape=(2048,)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Dense (200, activation='linear'))
    model.add(BatchNormalization())

    model.add(Activation('softmax')) # Transformar la salida de las neuronas

    model.summary()

    print()
    
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=SGD(), 
                  metrics=['accuracy'])

    # Entrenamiento del modelo
    hist = model.fit(train_pred, y_train, batch_size=32, verbose=1, epochs=20, 
                     validation_split=0.2)

    # Se prueba el modelo con el conjunto test
    predictions = model.predict(test_pred, verbose=1)

    # Se muestran los resultados
    acc = calcularAccuracy(y_test, predictions)

    mostrarEvolucion(hist)

    print("\nAccuracy: %f %%" % (acc*100))

# Reentrenar ResNet50 (fine tunning)
def Reentrenar(x_train, y_train, x_test, y_test):

    print("Reentrenar ResNet50 (fine tunning)")

    # Preprocesamiento de los conjuntos de datos de train y test
    x_train = preprocess_input(x_train)
    x_test = preprocess_input(x_test)

    # Modelo ResNet50 (preentrenado en ImageNet y sin la última capa).
    resnet50 = ResNet50(include_top=False, weights='imagenet', pooling='avg')

    # Modelo ResNet50 con nueva capa totalmente conectada
    x = resnet50.output
    x = Dense(2048, activation='relu')(x)
    x = BatchNormalization()(x)
    last = Dense(200, activation='softmax')(x)
    model = Model(inputs=resnet50.input, outputs=last)

    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=SGD(), 
                  metrics=['accuracy'])

    # Entrenamiento del modelo
    hist = model.fit(x_train, y_train, batch_size=32, verbose=1, epochs=14, 
                      validation_split=0.2)

    # Se prueba el modelo con el conjunto test
    predictions = model.predict(x_test, verbose=1)

    # Se muestran los resultados
    acc = calcularAccuracy(y_test, predictions)

    mostrarEvolucion(hist)

    print("\nAccuracy: %f %%" % (acc*100))

#########################################################################
################################# MAIN ##################################
#########################################################################
def main ():

    # Se cargan las imagenes y se dividen en conjuntos de train y test
    x_train, y_train, x_test, y_test = cargarDatos('/content')

    # Usar ResNet50 preentrenada en ImageNet como un extractor de características 
    ExtractorCaracteristicas (x_train, y_train, x_test, y_test)

    input('\nPulsar \'enter\' para continuar con el apartado 2:')

    # Reentrenar ResNet50 (fine tunning)
    Reentrenar(x_train, y_train, x_test, y_test)

#########################################################################
#########################################################################
    
# Se llama al programa principal para que ejecute el programa
if __name__ == '__main__':

    main()
